<img src='NEFUlogo.png' align='left'>

# Experiment of Identification of Query Intent based on Query Log.
<img src = 'classification.jpg'>

## Fire up graphlab

In [1]:
import graphlab

In [2]:
graphlab.canvas.set_target('ipynb')

## Load data

In [3]:
data = graphlab.SFrame.read_csv('Sogou_Extraction_Features_new.csv')

This non-commercial license of GraphLab Create is assigned to guoxiuhe@nefu.edu.cn and will expire on April 02, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-05-05 08:19:35,066 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: C:\Users\heguoxiu\AppData\Local\Temp\graphlab_server_1462407554.log.0


Finished parsing file E:\Python\2Graduation-Project\Sogou_Extraction_Features_new.csv

Parsing completed. Parsed 100 lines in 0.296401 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,long,str,str,str,list,long,long,str,dict,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file E:\Python\2Graduation-Project\Sogou_Extraction_Features_new.csv

Parsing completed. Parsed 10000 lines in 0.1248 secs.

## have a show of data

In [4]:
data.head()

﻿time,user_id,query,result_click,url,result_and_click,result_rank
00:00:00,2982199073774412,360安全卫士,8 3,download.it.com.cn/softweb/software/firewall/a ...,"[8, 3]",8
00:00:00,7594220010824798,哄抢救灾物资,1 1,news.21cn.com/social/daqian/2008/05/29/4777194 ...,"[1, 1]",1
00:00:00,5228056822071097,75810部队,14 5,www.greatoo.com/greatoo_cn/list.asp?link_id=27 ...,"[14, 5]",14
00:00:00,6140463203615646,绳艺,62 36,www.jd-cd.com/jd_opus/xx/200607/706.html ...,"[62, 36]",62
00:00:00,8561366108033201,汶川地震原因,3 2,www.big38.net,"[3, 2]",3
00:00:00,23908140386148713,莫衷一是的意思,1 2,www.chinabaike.com/article/81/82/110/2007/2007 ...,"[1, 2]",1
00:00:00,1797943298449139,星梦缘全集在线观��� ...,8 5,www.6wei.net/dianshiju/????\xa1\xe9|????do=index ...,"[8, 5]",8
00:00:00,717725924582846,闪字吧,1 2,www.shanziba.com,"[1, 2]",1
00:00:00,41416219018952116,霍震霆与朱玲玲照��� ...,2 6,bbs.gouzai.cn/thread-698736.html ...,"[2, 6]",2
00:00:00,9975666857142764,电脑创业,2 2,ks.cn.yahoo.com/question/1307120203719.html ...,"[2, 2]",2


## Caculate the query_word_count of the query_cut

In [5]:
data['query_word_count'] = graphlab.text_analytics.count_words(data['query_cut'])

In [6]:
data['query_search_word_count'] = graphlab.text_analytics.count_words(data['query_search_cut'])

In [7]:
data['tfidf'] = graphlab.text_analytics.tf_idf(data['query_word_count'])

In [8]:
data['search_tfidf'] = graphlab.text_analytics.tf_idf(data['query_search_word_count'])

## Set the train_data, validation_data, test_data

In [9]:
train_data, test_data = data.random_split(.8, seed=0)

In [10]:
train_data, validation_data = train_data.random_split(0.75, seed=0)

# Experiment Analysis

## 1. logistic_classifier

In [11]:
logistic_model_1 = graphlab.logistic_classifier.create(train_data, target='label', features=['tfidf', 'result_rank', 'click_rank'], validation_set=validation_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 3

...

| 5         | 8        | 1.000000  | 1.436021     | 0.858902          | 0.674085            |

| 6         | 9        | 1.000000  | 1.498025     | 0.864559          | 0.671118            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [12]:
logistic_model_2 = graphlab.logistic_classifier.create(train_data, target='label', features=['search_tfidf', 'result_rank', 'click_rank'], validation_set=validation_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 3

...

| 5         | 8        | 1.000000  | 0.256213     | 0.859235          | 0.669139            |

| 6         | 9        | 1.000000  | 0.310216     | 0.863894          | 0.665183            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [13]:
logistic_model_1.evaluate(test_data)

{'accuracy': 0.6610772357723578,
 'auc': 0.725856209091658,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 9
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |   Resource   |    Navigation   |   15  |
 |  Navigation  |    Navigation   |  108  |
 |  Navigation  |   Information   |  100  |
 | Information  |    Navigation   |   74  |
 | Information  |     Resource    |  197  |
 |  Navigation  |     Resource    |   13  |
 |   Resource   |     Resource    |  132  |
 |   Resource   |   Information   |  268  |
 | Information  |   Information   |  1061 |
 +--------------+-----------------+-------+
 [9 rows x 3 columns],
 'f1_score': 0.5446845242980177,
 'log_loss': 1.4042001737368937,
 'precision': 0.5588885064524872,
 'recall': 0.5344355395028676,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 


In [14]:
logistic_model_2.evaluate(test_data)

{'accuracy': 0.6590447154471545,
 'auc': 0.7228902472155562,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 9
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |   Resource   |    Navigation   |   13  |
 |  Navigation  |    Navigation   |  104  |
 |  Navigation  |   Information   |  103  |
 | Information  |    Navigation   |   76  |
 | Information  |     Resource    |  188  |
 |  Navigation  |     Resource    |   14  |
 |   Resource   |     Resource    |  125  |
 |   Resource   |   Information   |  277  |
 | Information  |   Information   |  1068 |
 +--------------+-----------------+-------+
 [9 rows x 3 columns],
 'f1_score': 0.5358960021326273,
 'log_loss': 1.349583001135879,
 'precision': 0.5528973871141066,
 'recall': 0.5245316187910093,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 


## 2. boosted_trees_classifier

In [15]:
boost_model_1 = graphlab.boosted_trees_classifier.create(train_data, target='label', features=['tfidf', 'result_rank', 'click_rank'], validation_set=validation_data)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 3

...

| 3         | 0.233410     | 0.712646          | 0.810148          | 0.686944            | 0.829215            |

| 4         | 0.285413     | 0.715308          | 0.771661          | 0.685955            | 0.796263            |

| 5         | 0.326416     | 0.714642          | 0.744124          | 0.688427            | 0.774047            |

| 6         | 0.376418     | 0.713478          | 0.721915          | 0.689416            | 0.755853            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [16]:
boost_model_2 = graphlab.boosted_trees_classifier.create(train_data, target='label', features=['search_tfidf', 'result_rank', 'click_rank'], validation_set=validation_data)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 3

...

| 3         | 0.122205     | 0.701165          | 0.839892          | 0.685955            | 0.855486            |

| 4         | 0.163208     | 0.702163          | 0.806195          | 0.686944            | 0.825325            |

| 5         | 0.211210     | 0.703328          | 0.784135          | 0.686449            | 0.806564            |

| 6         | 0.251213     | 0.703827          | 0.767120          | 0.687933            | 0.792809            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [17]:
boost_model_1.evaluate(test_data)

{'accuracy': 0.7032520325203252,
 'auc': 0.7466386443497933,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 8
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 | Information  |     Resource    |   8   |
 | Information  |    Navigation   |   36  |
 |  Navigation  |    Navigation   |   67  |
 |  Navigation  |   Information   |  154  |
 |   Resource   |    Navigation   |   17  |
 |   Resource   |     Resource    |   29  |
 |   Resource   |   Information   |  369  |
 | Information  |   Information   |  1288 |
 +--------------+-----------------+-------+
 [8 rows x 3 columns],
 'f1_score': 0.446959856678348,
 'log_loss': 0.7197325088756489,
 'precision': 0.6844421312532853,
 'recall': 0.4466713019512452,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 
 Data:
 +-----------+-----+-----+------+----

In [18]:
boost_model_2.evaluate(test_data)

{'accuracy': 0.695630081300813,
 'auc': 0.7024469874402678,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 7
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  Navigation  |    Navigation   |   27  |
 |  Navigation  |   Information   |  194  |
 | Information  |     Resource    |   10  |
 | Information  |    Navigation   |   8   |
 |   Resource   |     Resource    |   28  |
 |   Resource   |   Information   |  387  |
 | Information  |   Information   |  1314 |
 +--------------+-----------------+-------+
 [7 rows x 3 columns],
 'f1_score': 0.38297882964640456,
 'log_loss': 0.7596077955655148,
 'precision': 0.733891456874376,
 'recall': 0.39204277056863873,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 
 Data:
 +-----------+-----+-----+------+-----+-------+
 | threshold | fpr | tpr |  p   

## 3. decision_tree_classifier

In [19]:
tree_model_1 = graphlab.decision_tree_classifier.create(train_data, target='label', features=['tfidf', 'result_rank', 'click_rank'], validation_set=validation_data)

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 3

...

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| Iteration | Elapsed Time | Training-accuracy | Training-log_loss | Validation-accuracy | Validation-log_loss |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| 1         | 0.052201     | 0.706323          | 0.956458          | 0.686449            | 0.963650            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [20]:
tree_model_2 = graphlab.decision_tree_classifier.create(train_data, target='label', features=['search_tfidf', 'result_rank', 'click_rank'], validation_set=validation_data)

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 3

...

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| Iteration | Elapsed Time | Training-accuracy | Training-log_loss | Validation-accuracy | Validation-log_loss |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| 1         | 0.103803     | 0.695674          | 0.968638          | 0.674580            | 0.974970            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [21]:
tree_model_1.evaluate(test_data)

{'accuracy': 0.6966463414634146,
 'auc': 0.6867636034091679,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 9
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 | Information  |     Resource    |   8   |
 | Information  |    Navigation   |   22  |
 |  Navigation  |    Navigation   |   39  |
 |  Navigation  |   Information   |  181  |
 |   Resource   |    Navigation   |   9   |
 |  Navigation  |     Resource    |   1   |
 |   Resource   |     Resource    |   30  |
 |   Resource   |   Information   |  376  |
 | Information  |   Information   |  1302 |
 +--------------+-----------------+-------+
 [9 rows x 3 columns],
 'f1_score': 0.4054149847804111,
 'log_loss': 0.9634419449245485,
 'precision': 0.6755833909680063,
 'recall': 0.4087457407797592,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 

In [22]:
tree_model_2.evaluate(test_data)

{'accuracy': 0.6844512195121951,
 'auc': 0.6279185185171681,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 9
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |   Resource   |    Navigation   |   16  |
 |  Navigation  |    Navigation   |   21  |
 |  Navigation  |   Information   |  197  |
 | Information  |     Resource    |   9   |
 | Information  |    Navigation   |   11  |
 |  Navigation  |     Resource    |   3   |
 |   Resource   |     Resource    |   14  |
 |   Resource   |   Information   |  385  |
 | Information  |   Information   |  1312 |
 +--------------+-----------------+-------+
 [9 rows x 3 columns],
 'f1_score': 0.3443390296721847,
 'log_loss': 0.9739365547361191,
 'precision': 0.5562251238729591,
 'recall': 0.37124751639280335,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003


## 4. nearest_neighbor_classifier

In [23]:
knn_model_1 = graphlab.nearest_neighbor_classifier.create(train_data, target='label', features=['tfidf', 'result_rank', 'click_rank'])

Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

In [24]:
knn_model_2 = graphlab.nearest_neighbor_classifier.create(train_data, target='label', features=['search_tfidf', 'result_rank', 'click_rank'])

Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

In [25]:
knn_model_1.evaluate(test_data)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1968    | 0.0166389   | 22.001ms     |

...

+--------------+---------+-------------+--------------+

| 0            | 1968    | 0.0166389   | 0us          |

| Done         |         | 100         | 775.618ms    |

+--------------+---------+-------------+--------------+

{'accuracy': 0.6478658536585366, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 9
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 | Information  |   Information   |  1172 |
 |   Resource   |   Information   |  352  |
 |   Resource   |    Navigation   |   18  |
 | Information  |     Resource    |   93  |
 | Information  |    Navigation   |   67  |
 |  Navigation  |   Information   |  147  |
 |  Navigation  |    Navigation   |   58  |
 |  Navigation  |     Resource    |   16  |
 |   Resource   |     Resource    |   45  |
 +--------------+-----------------+-------+
 [9 rows x 3 columns]}

In [26]:
knn_model_2.evaluate(test_data)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1968    | 0.0166389   | 5ms          |

...

| 0            | 1968    | 0.0166389   | 15.6ms       |

| 1964         | 1e+007  | 99.817      | 1.03s        |

| Done         |         | 100         | 1.03s        |

+--------------+---------+-------------+--------------+

{'accuracy': 0.6590447154471545, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 9
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 | Information  |   Information   |  1236 |
 |   Resource   |   Information   |  368  |
 |  Navigation  |   Information   |  193  |
 | Information  |    Navigation   |   34  |
 | Information  |     Resource    |   62  |
 |   Resource   |     Resource    |   39  |
 |  Navigation  |     Resource    |   6   |
 |  Navigation  |    Navigation   |   22  |
 |   Resource   |    Navigation   |   8   |
 +--------------+-----------------+-------+
 [9 rows x 3 columns]}

## 5. random_forest_classifier

In [27]:
rf_model_1 = graphlab.random_forest_classifier.create(train_data, target='label', features=['tfidf', 'result_rank', 'click_rank'], validation_set=validation_data)

Random forest classifier:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 3

...

| 3         | 0.128205     | 0.694010          | 0.774196          | 0.673096            | 0.802446            |

| 4         | 0.157207     | 0.696007          | 0.770874          | 0.674580            | 0.798738            |

| 5         | 0.208210     | 0.696506          | 0.770847          | 0.677052            | 0.798629            |

| 6         | 0.249212     | 0.696672          | 0.770265          | 0.679031            | 0.796616            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [28]:
rf_model_2 = graphlab.random_forest_classifier.create(train_data, target='label', features=['search_tfidf', 'result_rank', 'click_rank'], validation_set=validation_data)

Random forest classifier:

--------------------------------------------------------

Number of examples          : 6010

Number of classes           : 3

Number of feature columns   : 3

...

| 3         | 0.175407     | 0.696007          | 0.796646          | 0.686449            | 0.814381            |

| 4         | 0.214409     | 0.693844          | 0.797727          | 0.684471            | 0.815634            |

| 5         | 0.260412     | 0.691847          | 0.796725          | 0.683976            | 0.814457            |

| 6         | 0.305414     | 0.694509          | 0.797516          | 0.685955            | 0.815020            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [29]:
rf_model_1.evaluate(test_data)

{'accuracy': 0.695630081300813,
 'auc': 0.7161717094073611,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 7
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  Navigation  |    Navigation   |   23  |
 |  Navigation  |   Information   |  198  |
 |   Resource   |     Resource    |   24  |
 |   Resource   |   Information   |  391  |
 | Information  |   Information   |  1322 |
 | Information  |    Navigation   |   6   |
 | Information  |     Resource    |   4   |
 +--------------+-----------------+-------+
 [7 rows x 3 columns],
 'f1_score': 0.3692155416296303,
 'log_loss': 0.7863180472972336,
 'precision': 0.7806769038296132,
 'recall': 0.38479873866124753,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 
 Data:
 +-----------+-----+-----+------+-----+-------+
 | threshold | fpr | tpr |  p   

In [30]:
rf_model_2.evaluate(test_data)

{'accuracy': 0.6941056910569106,
 'auc': 0.6590101441200914,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 7
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  Navigation  |    Navigation   |   18  |
 |  Navigation  |   Information   |  203  |
 |   Resource   |     Resource    |   24  |
 |   Resource   |   Information   |  391  |
 | Information  |   Information   |  1324 |
 | Information  |    Navigation   |   6   |
 | Information  |     Resource    |   2   |
 +--------------+-----------------+-------+
 [7 rows x 3 columns],
 'f1_score': 0.35685051456480027,
 'log_loss': 0.811039754646192,
 'precision': 0.7877931071361729,
 'recall': 0.37775776103203457,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 
 Data:
 +-----------+-----+-----+------+-----+-------+
 | threshold | fpr | tpr |  p  

# The END